# Data Scarping on HKJC


## Get date from hkjc

In [ ]:
driver = webdriver.Chrome()

time.sleep(2)


# try:
time.sleep(0.5)
driver.get(f"https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx")
select = Select(driver.find_element(By.ID, 'selectId'))

# Collect the values or visible text of all options
dates_extract = [option.text for option in select.options]

driver.quit()

dates_extract=set(dates_extract)
date_diff=dates_extract

try:
    with open("date_list.txt", "r") as file:
        lines = file.readlines()
        dates_old=[line.strip("\n") for line in lines]
        dates_old=set(dates_old)
        date_diff=dates_extract.difference(dates_old)
        date_diff=sorted(date_diff, key=lambda x: datetime.strptime(x, '%d/%m/%Y'),reverse=True)
    print("Records of date found in your storage, new dates will update to [date_list.txt]")
    os.remove('date_list.txt')
except:
    print("no records of date exist in your storage")
    pass

dates_extract=sorted(dates_extract, key=lambda x: datetime.strptime(x, '%d/%m/%Y'),reverse=True)

for i in dates_extract:
    with open("date_list.txt", "a") as f:
        f.write(i+"\n")

if len(date_diff)==0:
    print("No date need to be added")
else:
    print("Date extracted successfully from HKJC, stored to file name: [date_list.txt]")



## Get links from race record

In [ ]:
# Check if old URLS exists
try:
    old_link=np.loadtxt("link.txt", delimiter=",",dtype=str)
    old_link = [re.sub(r'^[\{\s\'"]+|[\}\s\'"]+$', "", url) for url in old_link]
    os.remove("link.txt")
except:
    old_link=np.array([])

# Initialize the Chrome driver
driver = webdriver.Chrome()

links_new = []
for Date in date_diff:
    for race in range(1, 13):  # extract 1-12 races
        try:
            time.sleep(0.5)
            driver.get(f"https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate={Date}&RaceNo={race}")

            # Wait until the table is present
            WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="innerContent"]/div[2]/div[5]/table'))
            )

            race_table = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[5]/table')
            rows = race_table.find_elements(By.TAG_NAME, 'tr')

            # Loop through each row to extract links
            for row in rows:
                # Find all anchor tags within the row
                anchors = row.find_elements(By.TAG_NAME, 'a')
                for anchor in anchors:
                    # Get the href attribute
                    link = anchor.get_attribute('href')
                    if link[-4:] !=".jpg":
                        links_new.append(link)

        except Exception as e:
            continue

# Close the driver after finishing the scraping
driver.quit()

links_new = [re.sub(r'^[\{\s\'"]+|[\}\s\'"]+$', "", url) for url in links_new]

#combine old links and new links
save_link=(set(old_link) | set(links_new))

#Update the links record
with open("link.txt", "w") as file:
   file.write(str(save_link))

## Grouping URLS by Horse Jockey and Trianer

In [ ]:
# Initialize lists to store full URLs
horse_urls = []
trainer_urls = []
jockey_urls = []

# Regular expressions to find IDs
horse_pattern = re.compile(r'HorseId=([^&]+)')
trainer_pattern = re.compile(r'TrainerId=([^&]+)')
jockey_pattern = re.compile(r'JockeyId=([^&]+)')

# Iterate through URLs and categorize them
for url in links_new:
    if horse_pattern.search(url):
        horse_urls.append(url)
    elif trainer_pattern.search(url):
        trainer_urls.append(url)
    elif jockey_pattern.search(url):
        jockey_urls.append(url)
        
horse_urls=set(horse_urls)
trainer_urls=set(trainer_urls)
jockey_urls=set(jockey_urls)

## Get Horse Record be each horse

In [ ]:
driver2 = webdriver.Chrome()

#Extract old record if exists
try:
    horseRecorddf=pd.read_csv('horseRecord.csv')
    horseRecorddf=horseRecorddf.reset_index(drop=True)
    print("Horse record found, updating record to [horseRecord.csv]")
except:
    print("Getting Horse Record, Record will be save into [horseRecord.csv]")
    horseRecorddf=pd.DataFrame()
    
for i in horse_urls:    
    try:

            time.sleep(1)
            driver2.get(i)

            name = driver2.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[1]/table/tbody/tr[1]/td/span')


            for idx in [3,4,5]:
                table_xpath = f'//*[@id="innerContent"]/div[2]/div[1]/table[{idx}]'
                try:
                    table_element = driver2.find_element(By.XPATH, table_xpath)
                    table_html = table_element.get_attribute('outerHTML')
                    df_list = pd.read_html(table_html)
                    race_record_df = df_list[0]
                    # Check for the "Race Index" column which is present in the English site
                    if "Race Index" in race_record_df.iloc[0,0]:
                        horsedf=race_record_df
                        break
                except Exception as e:
                    continue

            # Use XPath to find the <td> elements
            td_elements = driver2.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr/td')

            # Extract the text from the first <td> elements
            results = [td.text for td in td_elements]
            # Extract the text
            name_text = name.text

            data_dict = {}
            for i in range(0, len(results), 3):
                key = results[i].strip()  # Remove any leading/trailing whitespace
                value = results[i + 2].strip()  # Value is two indices ahead
                data_dict[key] = value



            horsedf.insert(1,"Colour / Sex", list(data_dict.values())[1], allow_duplicates=True)
            horsedf.insert(1,"Country of Origin / Age", list(data_dict.values())[0], allow_duplicates=True)
            horsedf.insert(1,"HorseName", name_text, allow_duplicates=True)
            try:
                horsedf.columns = horsedf.columns.astype(str)
                horseRecorddf=pd.concat([horseRecorddf,horsedf],ignore_index=True)
            except NameError:
                horseRecorddf=horsedf

    except:
        continue

# Close the driver after finishing the scraping
driver2.quit()


try:
    os.remove('horseRecord.csv')
    
except FileNotFoundError:
    pass



horseRecorddf.to_csv("horseRecord.csv",index=False)

print("Horse Record have been save into [horseRecord.csv]")

# Cleaning Horse record

In [ ]:
horserecord=pd.read_csv("horseRecord.csv")

In [ ]:
def to_ms(t):
    part = t.split(".")
    m = int(part[0])
    sec = int(part[1])
    ms = int(part[2])
    return ((m*60*1000)+(sec*1000)+ms)/1000


cleanedRecord = horserecord.copy()
cleanedRecord = cleanedRecord.rename(columns=horserecord.iloc[0]).copy()
cleanedRecord = cleanedRecord.rename(columns={cleanedRecord.columns[1]: "HorseName"})
cleanedRecord = cleanedRecord.rename(columns={cleanedRecord.columns[2]: "Country of Origin / Age"})
cleanedRecord = cleanedRecord.rename(columns={cleanedRecord.columns[3]: "Colour / Sex"})

cleanedRecord = cleanedRecord.dropna(subset=['Race Index'])

cleanedRecord['Pla.']=pd.to_numeric(cleanedRecord['Pla.'],errors='coerce',downcast="signed")
cleanedRecord['Dr.'] = pd.to_numeric(cleanedRecord['Dr.'],errors='coerce',downcast="signed")
cleanedRecord['Rtg.'] = pd.to_numeric(cleanedRecord['Rtg.'],errors='coerce',downcast="signed")
cleanedRecord['Act. Wt.'] = pd.to_numeric(cleanedRecord['Act. Wt.'],errors='coerce',downcast="signed")
cleanedRecord['Declar. Horse Wt.'] = pd.to_numeric(cleanedRecord['Declar. Horse Wt.'],errors='coerce')
cleanedRecord['Win Odds'] = pd.to_numeric(cleanedRecord['Win Odds'],errors='coerce')
cleanedRecord['Race Index']=pd.to_numeric(cleanedRecord['Race Index'],errors='coerce',downcast="signed")


cleanedRecord = cleanedRecord.dropna(subset=['Race Index'])
cleanedRecord['Race Index'] = cleanedRecord['Race Index'].astype('str')
cleanedRecord = cleanedRecord.dropna(subset=['Pla.'])
# cleanedRecord = cleanedRecord.drop('Video Replay',axis=1)
cleanedRecord = cleanedRecord[cleanedRecord['Race Class']!='GRIFFIN']


cleanedRecord['Rtg.'] = cleanedRecord['Rtg.'].fillna(value=100)
cleanedRecord[['RC','Track','Course']] = cleanedRecord['RC/Track/ Course'].str.split(" / ",expand=True)
cleanedRecord[['Country of Origin','Age']]=cleanedRecord["Country of Origin / Age"].str.split(" / ",expand=True)
cleanedRecord['Age']=pd.to_numeric(cleanedRecord['Age'],errors='coerce',downcast="signed")


split_cs = cleanedRecord["Colour / Sex"].str.split(" / ", expand=True)
cleanedRecord['Colour'] = split_cs[0]
cleanedRecord['Sex'] = split_cs[1]

cleanedRecord['Course']=cleanedRecord['Course'].str.replace('"','')
cleanedRecord['Finish Time'] = cleanedRecord['Finish Time'].apply(to_ms)


cleanedRecord=cleanedRecord.drop_duplicates(subset=["HorseName", "Date"])
gamecounts = dict(cleanedRecord['HorseName'].value_counts())
cleanedRecord['Total Raced'] = cleanedRecord['HorseName'].map(gamecounts)
wins = cleanedRecord.groupby('HorseName')['Pla.'].apply(lambda x: (x == 1).sum())
winrate = (wins / cleanedRecord.groupby('HorseName')['Total Raced'].first())*100
cleanedRecord['Win Rate'] = cleanedRecord['HorseName'].map(winrate)


cleanedRecord = cleanedRecord[cleanedRecord['Race Class']!='R']
cleanedRecord= cleanedRecord.drop(['RC/Track/ Course','Running Position','LBW','Total Raced',
                                   'Country of Origin / Age','Colour / Sex','Video Replay'],axis=1)
cleanedRecord=cleanedRecord.reset_index(drop=True)


cleanedRecord["Date"]=pd.to_datetime(cleanedRecord.Date)
cleanedRecord["Date"]=cleanedRecord["Date"].dt.strftime("%Y-%m-%d")


firstplace_df = cleanedRecord[cleanedRecord['Pla.'] == 1]

class5=cleanedRecord[cleanedRecord["Race Class"]=="5"]
class4=cleanedRecord[cleanedRecord["Race Class"]=="4"]
class3=cleanedRecord[cleanedRecord["Race Class"]=="3"]
class2=cleanedRecord[cleanedRecord["Race Class"]=="2"]
class1=cleanedRecord[cleanedRecord["Race Class"]=="1"]


In [ ]:
cleanedRecord.to_csv("cleanedRecord.csv",index=False)